<a href="https://colab.research.google.com/github/andrewgcodes/blend/blob/main/blend_song_vectors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Song2Vec

## (Rihanna + Chopin) / 2 = ???

This Colab notebook lets you find out what song is perfectly in-between two specified songs. For instance, what song is a blend of Only Girl in the World (Rihanna) and Nocturne Op 9 No 2 (Chopin).

This code uses the DISCO-200K music embeddings dataset and computes the mean of your two specified songs' embedding vectors. Then, it computes Euclidean distance with each of the 200k songs in the database and determines which songs are closest to the mean.

**You need to turn on the Colab GPU**

https://arxiv.org/abs/2306.13512

Imports

WAIT: Activate the GPU in Colab!

In [54]:
import torch
from queue import PriorityQueue
from torch.nn import functional as F
from datasets import load_dataset
import pickle

In [ ]:
!pip install datasets

Load the DISCO-200K high quality dataset from Hugging Face Datasets (about 1 Gigabyte)

There is a DISCO-10M available with 10 million songs! If you have the capacity to handle this, swap out "DISCO-200k-high-quality" with "DISCO-10M"

https://huggingface.co/datasets/DISCOX/DISCO-10M

In [57]:
# Load the dataset
ds = load_dataset("DISCOX/DISCO-200K-high-quality")

train_ds = ds['train']

  0%|          | 0/1 [00:00<?, ?it/s]

Load in inverted index and tensor embeddings (download from Hugging Face: https://huggingface.co/datasets/gaodrew/DISCO-200K-Tensors/tree/main)

In [63]:
with open('inverted_index.pkl', 'rb') as f:
    inverted_index = pickle.load(f)

all_embeddings = torch.load('all_embeddings.pt')

all_embeddings = F.normalize(all_embeddings, p=2, dim=1)

Functions

In [ ]:

def find_songs(query, num_matches=15):
    query_words = query.lower().split()
    match_indices = {}
    for word in query_words:
        for idx in inverted_index.get(word, []):
            match_indices.setdefault(idx, 0)
            match_indices[idx] += 1  # increment the count for this index

    # sort indices by match count (descending) and convert back to list
    sorted_indices = sorted(match_indices, key=match_indices.get, reverse=True)
    matches = [train_ds[i] for i in sorted_indices]
    return matches[:num_matches]


def select_song(matches):
    print("\nPlease select a song:")
    for i, song in enumerate(matches):
        print(f"{i}: {song['video_title_youtube']}")
    selection = int(input("Enter the number of your selection: "))
    return matches[selection]


Run this cell and type in two keywords (artist name, short song title). Then select the available songs.
Many songs are not available (we only have 200k)! Use the 10M dataset if you can.

In [68]:
# Define the video title queries
query1 = input("Song 1 title: ")
query2 = input("Song 2 title: ")

matches1 = find_songs(query1)
matches2 = find_songs(query2)

song1 = select_song(matches1)
song2 = select_song(matches2)


# Compute average embedding
embedding1 = torch.tensor(song1['audio_embedding_spotify']).cuda()
embedding2 = torch.tensor(song2['audio_embedding_spotify']).cuda()
average_embedding = torch.tensor((embedding1 + embedding2) / 2).cuda()

# Normalize embeddings
average_embedding = F.normalize(average_embedding, p=2, dim=0)

# Compute cosine similarity
cosine_similarities = torch.matmul(all_embeddings, average_embedding)

# Get the top 5 songs
_, top_5_indices = torch.topk(cosine_similarities, 5)
top_5_indices = top_5_indices.cpu().numpy()

for idx in top_5_indices:
    print(train_ds[int(idx)]['video_title_youtube'])
    print(train_ds[int(idx)]['video_url_youtube'])



Song 1 title: violin
Song 2 title: jazz

Please select a song:
0: Albert Dietrich F-A-E Sonata for Violin and Piano - I. Allegro - Jason Issokson, Violin
1: 12 Violin Sonatas, Op. 5: XII. Violin Sonata in D Minor "La Follia" (Preludii - Allemanda -...
2: 6 Melodies for Violin & Keyboard (Arr. A. Larget-Caplan for Violin & Guitar) : No. 6, —
3: The Butterfly Lovers' Violin Concerto (Arr. for Violin & Chinese Orchestra) : Adagio cantabile...
4: J. S. Bach: Violin Partita No. 1 in B minor, Sarabande & Double // Cam Kjøll, violin
5: Violin Sonata in B Minor, Op. 31: II. Allegro
6: String Quintet in C Major, Op. 50 No. 3, G. 374 (Arr. E. Moreno & A. Zapico for Violin &...
7: Violin Sonata No. 8, "Sacra Spina"
8: Organ Concerto in D Minor, BWV 596 (arr. of Vivaldi's Violin Concerto in D Minor, RV 565) :...
9: Duet for Violin & Cello in F Major, Op. 6 No. 2: I. Andante
10: Elisabeth Jacquet de la Guerre- Violin Sonata No.1 in D minor
11: Le carnaval de Pesth, S379a (arr. of Hungarian Rhapsody

<ipython-input-68-6ee71fa97215>:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  average_embedding = torch.tensor((embedding1 + embedding2) / 2).cuda()
